Wagner Rogério de Andrade RA 036546

# Pré processamento
* Leia o arquivo Bias_correction_ucl.csv

In [3]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00514/Bias_correction_ucl.csv"
original_ko_temp_forecast_df = pd.read_csv(url)

* Remova a coluna "Next_Tmin".
* Remova a coluna Date

In [4]:
ko_temp_forecast_df = original_ko_temp_forecast_df.drop(['Next_Tmin', 'Date'], \
                                                        axis=1)

* Remova as linhas que tem valor faltante. Das 7752 linhas originais sobram 7588

In [5]:
ko_temp_forecast_df.shape

(7752, 23)

In [6]:
ko_temp_forecast_df.dropna(inplace=True)

In [7]:
ko_temp_forecast_df.shape

(7588, 23)

* O atributo de saída é Next_Tmax (a temperatura máxima no próximo dia). Vamos removê-lo do conjunto de dados para evitar contaminação durante o processo de normalização.


In [8]:
next_tmax = ko_temp_forecast_df['Next_Tmax']
ko_temp_forecast_df.drop('Next_Tmax', axis=1, inplace=True)

* Centralize e normalize cada atributo de entrada

In [60]:
from sklearn import preprocessing

prep_ko_temp_forecast_np = preprocessing.scale(ko_temp_forecast_df)

# Busca de hiperparâmetros e modelos utilizando:
* 5 fold cross validation como técnica de resampling separando os dados em conjuntos de 70% de treino e 30% de teste;
* RMSE como medida de erro;
* Busca aleatória de hiperparametros (nos modelos aplicáveis)

In [97]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
import numpy as np
import random

# Numero de vezes em que serão gerados valores aleatórios de hiperparâmetros 
# no teste
HYPERPARAM_SAMPLING_N = 10


# Calcula e returna o valor de RMSE score no conjunto de dados fornecido 
# usando o modelo especificado.
#
# X = conjunto de entrada (feature set)
# y = conjunto alvo (target ou class)
# cv = cross validator to be used by sklearn.model_selection
#      cross_val_score in order to determine the RMSE
#      default: ShuffleSplit with 5 fold on 70% test set
#
# Returns: RMSE value
#
# Reference: https://scikit-learn.org/stable/modules/cross_validation.html
def x_val_positive_rmse(model, X, y, cv = ShuffleSplit(\
                                                    n_splits=5, \
                                                    test_size=0.3)):
    scores = cross_val_score(model, \
                             X, \
                             y,
                             cv = ShuffleSplit(n_splits=5, test_size=0.3), \
                             scoring=('neg_root_mean_squared_error'))
    return np.mean(- scores)

# Calcula e retorna o valor do RMSE score da execução de um modelo usando 
# x_val_positive_rmse sem nenhum parâmetro especial bem como seu correspondente
# RMSE score melhor ajustado para HYPERPARAM_SAMPLING_N valores aleatoriamente
# escolhidos de hiperparâmetros conforme estipulado por tunned_model_lambda
#
# X = conjunto de entrada (feature set)
# y = conjunto alvo (target ou class)
# defaul_model = the default model 
# tunned_model_lambda = a lambda function that generates a model with tunned
#                       hyperparameters
#
# Returns:
#     default_params_RMSE: the RMSE value without any tunning parameter
#     best_params_RMSE the RMSE value with tunning parameters
def model_default_vs_tunned_compare(defaul_model, tunned_model_lambda, X, y):

    default_params_RMSE = x_val_positive_rmse(defaul_model, X, y)

    best_params_RMSE = 10**3
    for i in range(HYPERPARAM_SAMPLING_N):
        mean_iter_scores = x_val_positive_rmse(tunned_model_lambda(), X, y)

        if best_params_RMSE > mean_iter_scores:
            best_params_RMSE = mean_iter_scores  
    
    return default_params_RMSE, best_params_RMSE

## Linear

In [74]:
from sklearn.linear_model import LinearRegression

traditional_lr_rmse = x_val_positive_rmse(LinearRegression(), \
                                          prep_ko_temp_forecast_np,\
                                          next_tmax)

## Linear com regularização L2
Sendo alpha escolhido com: 10 números aleatórios entre 10^-3 e 10^3

In [98]:
from sklearn.linear_model import Ridge

ridge = lambda: Ridge(alpha=random.uniform(10**-3, 10**3))

default_ridge_lr_RMSE, best_ridge_lr_RMSE = \
    model_default_vs_tunned_compare(Ridge(), \
                                    ridge,\
                                    prep_ko_temp_forecast_np,\
                                    next_tmax)

## Linear com regularização L1
Sendo alpha escolhido com: 10 números aleatórios entre 10^-3 e 10^3

In [99]:
from sklearn.linear_model import Lasso

lasso = lambda: Lasso(alpha=random.uniform(10**-3, 10**3))

default_lasso_lr_RMSE, best_lasso_lr_RMSE = \
    model_default_vs_tunned_compare(Lasso(), \
                                    lasso, \
                                    prep_ko_temp_forecast_np,\
                                    next_tmax)

## SVM Linear
Sendo 10 amostras aleatórias com epsilon sendo 0.1 ou 0.3 e C entre 2^-5 e 2^15


In [83]:
from sklearn.svm import LinearSVR

svr_linear = lambda: LinearSVR(C = random.uniform(2**-5, 2**15), \
                               epsilon = random.choice([0.1, 0.3]))

default_svr_linear_RMSE, best_svr_linear_RMSE = \
    model_default_vs_tunned_compare(LinearSVR(), \
                                    svr_linear, \
                                    prep_ko_temp_forecast_np, \
                                    next_tmax)

## SVM com kernel RBF
Sendo 10 amostras aleatórias com epsilon sendo 0.1 ou 0.3, C entre 2^-5 e 2^15 e gamma entre 2^-9 e 2^3

In [84]:
from sklearn.svm import SVR

svr = lambda: SVR(epsilon = random.choice([0.1, 0.3]), \
                  C = random.uniform(2**-5, 2**15), \
                  gamma = random.uniform(2**-9, 2**3), \
                  kernel='rbf')

default_svr_rbf_RMSE, best_svr_rbf_RMSE = \
    model_default_vs_tunned_compare(SVR(kernel='rbf'), \
                                    svr, \
                                    prep_ko_temp_forecast_np, \
                                    next_tmax)

## KNN
Sendo K: 10 números aleatórios entre 1 e 1000

In [56]:
from sklearn.neighbors import KNeighborsRegressor

knn_regressor = lambda: KNeighborsRegressor(n_neighbors=random.randint(1, 1000))

default_knn_regr_RMSE, best_knn_regr_RMSE = \
    model_default_vs_tunned_compare(KNeighborsRegressor(), \
                                    knn_regressor, \
                                    prep_ko_temp_forecast_np,\
                                    next_tmax)

## MLP
Neurônios na camada do meio: de 5 a 20, de três em três

In [57]:
from sklearn.neural_network import MLPRegressor

#--------------------------------------------------------------------------
# Default hyperparameters
default_mlp_regr_RMSE = x_val_positive_rmse(MLPRegressor(), \
                                            prep_ko_temp_forecast_np,\
                                            next_tmax)

#--------------------------------------------------------------------------
# Tunning hyperparams

best_mlp_regr_RMSE = 10**3

neurons = 5

while neurons <= 20:
    mlp_regressor = MLPRegressor(hidden_layer_sizes=neurons)
    
    mean_iter_scores = x_val_positive_rmse(mlp_regressor, \
                                           prep_ko_temp_forecast_np, next_tmax)
    
    if best_mlp_regr_RMSE > mean_iter_scores:
        best_mlp_regr_RMSE = mean_iter_scores
    
    neurons +=3

## Arvore de decisão
* Usando prunning com ccp_alpha sendo 10 números aleatórios entre 0.0 e 0.04.


In [85]:
from sklearn.tree import DecisionTreeRegressor

dec_tree_regressor = lambda: DecisionTreeRegressor(ccp_alpha=random. \
                                                   uniform(0.0, 0.04))

default_dec_tree_RMSE, best_dec_tree_RMSE = \
    model_default_vs_tunned_compare(DecisionTreeRegressor(), \
                                    dec_tree_regressor, \
                                    prep_ko_temp_forecast_np, \
                                    next_tmax)

## Random Forest
Usando todas as combinações dos valores de n_estimators como 10, 100 e 1000 e max_features como 5, 10, e 22.

In [89]:
from sklearn.ensemble import RandomForestRegressor

#--------------------------------------------------------------------------
# Default hyperparameters
default_rand_for_RMSE = x_val_positive_rmse(\
                            RandomForestRegressor(), \
                            prep_ko_temp_forecast_np, \
                            next_tmax)

#--------------------------------------------------------------------------
# Tunning hyperparams

best_rand_for_RMSE = 10**3
n_estimators = [10, 100, 1000]
max_features = [5, 10, 22]

for estimator in n_estimators:
    for max_feature in max_features:
        dec_tree_regressor = RandomForestRegressor(n_estimators = estimator, \
                                                   max_features = max_feature)
        mean_iter_scores = x_val_positive_rmse(dec_tree_regressor, \
                                               prep_ko_temp_forecast_np,\
                                               next_tmax)
        
        if best_rand_for_RMSE > mean_iter_scores:
            best_rand_for_RMSE = mean_iter_scores

## GBM
Selecionando 10 trincas aleatórias de n_estimators entre 5 e 100, learning_rate entre 0.01 e 0.3 e max_depth: 2 ou 3

In [86]:
from sklearn.ensemble import GradientBoostingRegressor

gbm = lambda: GradientBoostingRegressor(\
                    max_depth = random.choice([2,3]), \
                    learning_rate = random.uniform(0.01, 0.3), \
                    n_estimators = random.randint(5, 100))

default_gbm_RMSE, best_gbm_RMSE = \
    model_default_vs_tunned_compare(GradientBoostingRegressor(), \
                                    gbm,\
                                    prep_ko_temp_forecast_np,\
                                    next_tmax)


# Tabela final
Tabela final com **cada classificador**, os **valores do RMSE** com valores default para os hiperparametros, e o **valor do RMSE** com o melhor valor dos hiperparametros.

In [100]:
columns = ['Regressor', 'RMSE default', 'RMSE melhores hiperparametros']
data = [
        ['Linear',            traditional_lr_rmse,    'N/A'], \
        ['Ridge',             default_ridge_lr_RMSE,   best_ridge_lr_RMSE], \
        ['Lasso',             default_lasso_lr_RMSE,   best_lasso_lr_RMSE], \
        ['SVR Linear',        default_svr_linear_RMSE, best_svr_linear_RMSE], \
        ['SVR RBF',           default_svr_rbf_RMSE,    best_svr_rbf_RMSE], \
        ['KNN',               default_knn_regr_RMSE,   best_knn_regr_RMSE], \
        ['MLP',               default_mlp_regr_RMSE,   best_mlp_regr_RMSE], \
        ['Arvore de decisao', default_dec_tree_RMSE,   best_dec_tree_RMSE], \
        ['Random Forest',     default_rand_for_RMSE,   best_rand_for_RMSE], \
        ['GBM',               default_gbm_RMSE,        best_gbm_RMSE], \
       ]

final_table = pd.DataFrame(data, columns = columns)

In [101]:
final_table

,Regressor,RMSE default,RMSE melhores hiperparametros
0,Linear,1.447540,N/A
1,Ridge,1.475565,1.440763
2,Lasso,1.975312,3.089508
3,SVR Linear,1.463359,1.810254
4,SVR RBF,1.199155,1.359733
5,KNN,1.270178,1.685001
6,MLP,1.393400,1.882743
7,Arvore de decisao,1.528704,1.425551
8,Random Forest,1.000956,0.940993
9,GBM,1.210508,1.276217
